# Setting up the AlexNet Model on CIFAR100

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import torch
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np

# Check for GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
# Update transform for 224x224 images
transform = transforms.Compose([
    transforms.Resize(224),  # Resize CIFAR-100 images to 224x224
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize images to [-1, 1]
])


In [ ]:
# Setting up Datasets
train_dataset = datasets.CIFAR100(root='dataset/', train=True, transform=transform, download=True)
test_dataset = datasets.CIFAR100(root='dataset/', train=False, transform=transform, download=True)

# Setting up Dataloaders
batch_size = 64
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
# Testing the Dataloader
def imshow(img):
    img = img / 2 + 0.5  # Unnormalize the image
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# Visualize some training images
dataiter = iter(train_loader)
images, labels = next(dataiter)

imshow(torchvision.utils.make_grid(images[:10]))
print(' '.join('%5s' % train_dataset.classes[labels[j]] for j in range(10)))

In [ ]:
print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of test samples: {len(test_dataset)}")
print(f"Classes: {train_dataset.classes}")

In [ ]:
# Import pre-trained AlexNet
model = torchvision.models.alexnet(pretrained=False)

# Modify the classifier for CIFAR-100 (100 classes)
model.classifier[6] = nn.Linear(4096, 100)

# Send model to the appropriate device
model = model.to(device)

In [ ]:
from tqdm import tqdm

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    # Wrap the data loader with tqdm for a progress bar
    with tqdm(train_loader, unit="batch") as tepoch:
        tepoch.set_description(f"Epoch {epoch+1}/{num_epochs}")

        for inputs, labels in tepoch:
            inputs, labels = inputs.to(device), labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            # Update tqdm with loss information
            tepoch.set_postfix(loss=(running_loss / (tepoch.n + 1)))

    print(f"Epoch {epoch+1}/{num_epochs} completed. Loss: {running_loss / len(train_loader):.4f}")

print("Training Finished!")

In [ ]:
# Evaluation loop
model.eval()
correct = 0
total = 0

with tqdm(test_loader, unit="batch", desc="Evaluating") as teval:
    with torch.no_grad():
        for inputs, labels in teval:
            inputs, labels = inputs.to(device), labels.to(device)

            # Forward pass
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # Update tqdm with accuracy
            teval.set_postfix(accuracy=(100 * correct / total))

print(f"Accuracy on the test set: {100 * correct / total:.2f}%")

In [4]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm

# Check for GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Data preprocessing and augmentation
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.Resize(224),  # Resize CIFAR-100 images to 224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet stats
])

transform_test = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load CIFAR-100 dataset
train_dataset = torchvision.datasets.CIFAR100(root='./data', train=True, transform=transform_train, download=True)
test_dataset = torchvision.datasets.CIFAR100(root='./data', train=False, transform=transform_test, download=True)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=2)

# Load pretrained AlexNet and modify it for CIFAR-100
model = torchvision.models.alexnet(pretrained=True)
model.classifier[6] = nn.Linear(4096, 100)  # Change the output layer to match CIFAR-100 classes
model = model.to(device)

# Freeze feature extractor layers
for param in model.features.parameters():
    param.requires_grad = False

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

# Training loop
num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    with tqdm(train_loader, unit="batch") as tepoch:
        tepoch.set_description(f"Epoch {epoch+1}/{num_epochs}")

        for inputs, labels in tepoch:
            inputs, labels = inputs.to(device), labels.to(device)

            # Zero gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            tepoch.set_postfix(loss=(running_loss / (tepoch.n + 1)))

    scheduler.step()
    print(f"Epoch {epoch+1}/{num_epochs} completed. Loss: {running_loss / len(train_loader):.4f}")

print("Training completed!")

# Evaluation loop
model.eval()
correct = 0
total = 0

with tqdm(test_loader, unit="batch", desc="Evaluating") as teval:
    with torch.no_grad():
        for inputs, labels in teval:
            inputs, labels = inputs.to(device), labels.to(device)

            # Forward pass
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # Update tqdm with accuracy
            teval.set_postfix(accuracy=(100 * correct / total))

print(f"Test Accuracy: {100 * correct / total:.2f}%")

Using device: cuda


100%|██████████| 169001437/169001437 [00:01<00:00, 102225239.00it/s]


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:01<00:00, 182MB/s]  
Epoch 1/30: 100%|██████████| 391/391 [01:11<00:00,  5.46batch/s, loss=2.28]


Epoch 1/30 completed. Loss: 2.2837


...

Epoch 29/30 completed. Loss: 0.7068


Epoch 30/30: 100%|██████████| 391/391 [01:00<00:00,  6.46batch/s, loss=0.706]


Epoch 30/30 completed. Loss: 0.7044
Training completed!


Evaluating: 100%|██████████| 79/79 [00:13<00:00,  5.99batch/s, accuracy=68.3]

Test Accuracy: 68.27%


In [5]:
# Save the trained model
model_path = "alexnet_cifar100.pth"
torch.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}")

Model saved to alexnet_cifar100.pth


# Patch Attacks Applied

In [8]:
# Load the model
model_path = ("/kaggle/input/alexnet-cifar100/pytorch/default/1/alexnet_cifar100.pth")
model = torchvision.models.alexnet(pretrained=False)  # Initialize AlexNet
model.classifier[6] = nn.Linear(4096, 100)  # Adjust for CIFAR-100
model.load_state_dict(torch.load(model_path))  # Load the saved state dictionary
print(device)
model = model.to(device)  # Send the model to the appropriate device
print("Model loaded successfully!")

/tmp/ipykernel_30/3071232291.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))  # Load the saved state dictionary


cuda
Model loaded successfully!


## Patch Attacks with 1% Accuracy

In [15]:
import numpy as np
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.autograd import Variable

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def patch_initialization(patch_type='rectangle', image_size=(3, 224, 224), noise_percentage=0.03):
    if patch_type == 'rectangle':
        mask_length = int((noise_percentage * image_size[1] * image_size[2])**0.5)
        mask_length = min(mask_length, image_size[1] - 1, image_size[2] - 1)  # Ensure it fits within the image
        patch = np.random.rand(image_size[0], mask_length, mask_length)
    print(f"Initialized patch of shape {patch.shape} for image size {image_size}.")  # Debugging output
    return patch

def mask_generation(mask_type='rectangle', patch=None, image_size=(3, 224, 224)):
    applied_patch = np.zeros(image_size)
    if mask_type == 'rectangle':
        rotation_angle = np.random.choice(4)
        for i in range(patch.shape[0]):
            patch[i] = np.rot90(patch[i], rotation_angle)

        max_x = image_size[1] - patch.shape[1]
        max_y = image_size[2] - patch.shape[2]

        if max_x <= 0 or max_y <= 0:
            raise ValueError(
                f"Patch size ({patch.shape}) exceeds image dimensions ({image_size}). "
                "Adjust `noise_percentage` or ensure patch is initialized correctly."
            )

        x_location, y_location = np.random.randint(low=0, high=max_x), np.random.randint(low=0, high=max_y)

        for i in range(patch.shape[0]):
            applied_patch[:, x_location:x_location + patch.shape[1], y_location:y_location + patch.shape[2]] = patch
    mask = applied_patch.copy()
    mask[mask != 0] = 1.0
    return applied_patch, mask, x_location, y_location

# Patch Attack
def patch_attack(image, applied_patch, mask, target, probability_threshold, model, lr=1, max_iteration=10):
    model.eval()
    
    # Ensure tensors are on the correct device
    applied_patch = torch.from_numpy(applied_patch).float().to(device)
    mask = torch.from_numpy(mask).float().to(device)
    image = image.to(device)

    target_probability, count = 0, 0
    perturbated_image = (mask * applied_patch) + ((1 - mask) * image)
    
    while target_probability < probability_threshold and count < max_iteration:
        count += 1
        perturbated_image = Variable(perturbated_image.data, requires_grad=True)

        output = model(perturbated_image.unsqueeze(0))  # Add batch dimension
        target_log_softmax = F.log_softmax(output, dim=1)[0][target]
        target_log_softmax.backward()

        patch_grad = perturbated_image.grad.clone()
        perturbated_image.grad.data.zero_()
        applied_patch += lr * patch_grad
        applied_patch = torch.clamp(applied_patch, min=-1, max=1)

        perturbated_image = (mask * applied_patch) + ((1 - mask) * image)
        perturbated_image = torch.clamp(perturbated_image, min=-1, max=1)

        output = model(perturbated_image.unsqueeze(0))
        target_probability = F.softmax(output, dim=1).data[0][target]

    return perturbated_image.cpu().numpy(), applied_patch.cpu().numpy()

# Display Patch
def display_patch(patch, epoch):
    mean, std = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]
    patch_visual = np.clip(np.transpose(patch, (1, 2, 0)) * std + mean, 0, 1)
    plt.imshow(patch_visual)
    plt.title(f"Patch Visualization - Epoch {epoch}")
    plt.axis('off')
    plt.savefig(f"patch_epoch_{epoch}.png")
    plt.show()

# Main Attack Loop
patch_type = 'rectangle'
patch = patch_initialization(patch_type, image_size=(3, 224, 224), noise_percentage=0.01)  # Try smaller percentages
epochs = 5
target = 7  # Target class index
probability_threshold = 0.7
lr = 1
max_iteration = 500

for epoch in range(epochs):
    train_success = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
    
        # Loop over the batch
        for i in range(images.size(0)):  # Dynamically handle batch size
            if i >= predicted.size(0):  # Ensure index is within bounds
                continue
    
            if predicted[i] != labels[i]:
                try:
                    applied_patch, mask, x_loc, y_loc = mask_generation(patch_type, patch, image_size=(3, 224, 224))
                    perturbated_image, patch = patch_attack(
                        images[i], applied_patch, mask, target, probability_threshold, model, lr, max_iteration
                    )
                    perturbated_image = torch.from_numpy(perturbated_image).to(device)
    
                    output = model(perturbated_image.unsqueeze(0))  # Add batch dimension
                    _, single_prediction = torch.max(output.data, 1)
    
                    if single_prediction.item() == target:
                        train_success += 1
                except ValueError as e:
                    print(f"Skipping iteration due to error: {e}")
    
    display_patch(patch, epoch)
    print(f"Epoch {epoch}: Success Rate on Train Set: {train_success / len(train_loader):.3f}")

Initialized patch of shape (3, 22, 22) for image size (3, 224, 224).
Skipping iteration due to error: Patch size ((3, 224, 224)) exceeds image dimensions ((3, 224, 224)). Adjust `noise_percentage` or ensure patch is initialized correctly.
Skipping iteration due to error: Patch size ((3, 224, 224)) exceeds image dimensions ((3, 224, 224)). Adjust `noise_percentage` or ensure patch is initialized correctly.
Skipping iteration due to error: Patch size ((3, 224, 224)) exceeds image dimensions ((3, 224, 224)). Adjust `noise_percentage` or ensure patch is initialized correctly.
Skipping iteration due to error: Patch size ((3, 224, 224)) exceeds image dimensions ((3, 224, 224)). Adjust `noise_percentage` or ensure patch is initialized correctly.
Skipping iteration due to error: Patch size ((3, 224, 224)) exceeds image dimensions ((3, 224, 224)). Adjust `noise_percentage` or ensure patch is initialized correctly.
Skipping iteration due to error: Patch size ((3, 224, 224)) exceeds image dimensi

KeyboardInterrupt: 

## Patch Attacks with 3% Accuracy

In [16]:
import numpy as np
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.autograd import Variable

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Patch Initialization
def patch_initialization(patch_type='rectangle', image_size=(3, 224, 224), noise_percentage=0.03):
    if patch_type == 'rectangle':
        mask_length = int((noise_percentage * image_size[1] * image_size[2])**0.5)
        patch = np.random.rand(image_size[0], mask_length, mask_length)
    return patch

# Mask Generation and Patch Application
def mask_generation(mask_type='rectangle', patch=None, image_size=(3, 224, 224)):
    applied_patch = np.zeros(image_size)
    if mask_type == 'rectangle':
        rotation_angle = np.random.choice(4)
        for i in range(patch.shape[0]):
            patch[i] = np.rot90(patch[i], rotation_angle)
        x_location, y_location = np.random.randint(low=0, high=image_size[1] - patch.shape[1]), \
                                 np.random.randint(low=0, high=image_size[2] - patch.shape[2])
        for i in range(patch.shape[0]):
            applied_patch[:, x_location:x_location + patch.shape[1], y_location:y_location + patch.shape[2]] = patch
    mask = applied_patch.copy()
    mask[mask != 0] = 1.0
    return applied_patch, mask, x_location, y_location

# Patch Attack
def patch_attack(image, applied_patch, mask, target, probability_threshold, model, lr=1, max_iteration=10):
    model.eval()
    
    # Ensure tensors are on the correct device
    applied_patch = torch.from_numpy(applied_patch).float().to(device)
    mask = torch.from_numpy(mask).float().to(device)
    image = image.to(device)

    target_probability, count = 0, 0
    perturbated_image = (mask * applied_patch) + ((1 - mask) * image)
    
    while target_probability < probability_threshold and count < max_iteration:
        count += 1
        perturbated_image = Variable(perturbated_image.data, requires_grad=True)

        output = model(perturbated_image.unsqueeze(0))  # Add batch dimension
        target_log_softmax = F.log_softmax(output, dim=1)[0][target]
        target_log_softmax.backward()

        patch_grad = perturbated_image.grad.clone()
        perturbated_image.grad.data.zero_()
        applied_patch += lr * patch_grad
        applied_patch = torch.clamp(applied_patch, min=-1, max=1)

        perturbated_image = (mask * applied_patch) + ((1 - mask) * image)
        perturbated_image = torch.clamp(perturbated_image, min=-1, max=1)

        output = model(perturbated_image.unsqueeze(0))
        target_probability = F.softmax(output, dim=1).data[0][target]

    return perturbated_image.cpu().numpy(), applied_patch.cpu().numpy()

# Display Patch
def display_patch(patch, epoch):
    mean, std = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]
    patch_visual = np.clip(np.transpose(patch, (1, 2, 0)) * std + mean, 0, 1)
    plt.imshow(patch_visual)
    plt.title(f"Patch Visualization - Epoch {epoch}")
    plt.axis('off')
    plt.savefig(f"patch_epoch_{epoch}.png")
    plt.show()

# Main Attack Loop
patch_type = 'rectangle'
patch = patch_initialization(patch_type, image_size=(3, 224, 224), noise_percentage=0.03)
epochs = 5
target = 7  # Target class index
probability_threshold = 0.7
lr = 1
max_iteration = 500

for epoch in range(epochs):
    train_success = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        
        for i in range(images.size(0)):
            if predicted[i] != labels[i]:
                applied_patch, mask, x_loc, y_loc = mask_generation(patch_type, patch, image_size=(3, 224, 224))
                perturbated_image, patch = patch_attack(
                    images[i], applied_patch, mask, target, probability_threshold, model, lr, max_iteration
                )
                perturbated_image = torch.from_numpy(perturbated_image).to(device)

                output = model(perturbated_image.unsqueeze(0))
                _, predicted = torch.max(output.data, 1)

                if predicted[0].item() == target:
                    train_success += 1

    display_patch(patch, epoch)
    print(f"Epoch {epoch}: Success Rate on Train Set: {train_success / len(train_loader):.3f}")

IndexError: index 5 is out of bounds for dimension 0 with size 1